## 02. Data Preprocessing: Feature Exclusion & Datetime Processing

**Objective:** Drop outcome-related, identifier, and redundant datetime features; parse and combine date and time into a single datetime column.

**PRD References:**
- 4.3 Features to Exclude
- 3.1.2 Data Cleaning & Preprocessing

**Notebook:** 02_data_preprocessing.ipynb (Part 1)

In [1]:
import pandas as pd
import numpy as np

# Load raw dataset
data_path = '../data/raw/RTA_EDSA_2007-2016.csv'
try:
    df = pd.read_csv(data_path)
    print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find data at {data_path}")

Loaded data: 22072 rows, 26 columns


### 1. Drop Outcome-Related Features

To avoid data leakage, we remove all casualty-related columns (`killed_*`, `injured_*`) and total counts, since our target will be defined separately.

In [2]:
# List of casualty and outcome-related columns to drop
outcome_cols = [
    'killed_driver', 'killed_passenger', 'killed_pedestrian',
    'injured_driver', 'injured_passenger', 'injured_pedestrian',
    'killed_uncategorized', 'injured_uncategorized',
    'killed_total', 'injured_total'
]
# Drop if present
df.drop(columns=[c for c in outcome_cols if c in df.columns], inplace=True)
print(f"After dropping outcome columns: {df.shape[1]} columns remain")

After dropping outcome columns: 16 columns remain


### 2. Drop Identifier and Redundant Columns

Remove identifiers and address fields, as well as redundant datetime PST column.

In [3]:
# Identifier, address, and redundant datetime columns
drop_cols = ['INCIDENTDETAILS_ID', 'LOCATION_TEXT', 'ADDRESS', 'DATETIME_PST']
# Drop only existing
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
print(f"After dropping identifiers & redundant datetime: {df.shape[1]} columns remain")

After dropping identifiers & redundant datetime: 12 columns remain


### 3. Parse and Combine Date and Time

Convert `DATE_UTC` and `TIME_UTC` into a single `DATETIME_UTC` column for downstream processing, then drop the originals.

In [4]:
# Parse DATETIME_UTC
df['DATETIME_UTC'] = pd.to_datetime(df['DATE_UTC'].astype(str) + ' ' + df['TIME_UTC'].astype(str), errors='coerce')

# Drop original date and time columns
for col in ['DATE_UTC', 'TIME_UTC']:
    if col in df.columns:
        df.drop(columns=col, inplace=True)

print(f"After parsing datetime: {df.shape[1]} columns remain")
print(df['DATETIME_UTC'].head())

After parsing datetime: 11 columns remain
0   2014-06-30 05:40:00
1   2014-03-17 01:00:00
2   2013-11-26 02:00:00
3   2013-10-26 13:00:00
4   2013-06-26 23:30:00
Name: DATETIME_UTC, dtype: datetime64[ns]


### 4. Temporal Feature Engineering

In this section, we extract various temporal features from the `DATETIME_UTC` column, including hour of day, day of week, month, year, season, and weekend flag.

In [7]:
# Extract temporal features from DATETIME_UTC
if 'df' in locals():
    df['hour'] = df['DATETIME_UTC'].dt.hour
    df['day_of_week'] = df['DATETIME_UTC'].dt.dayofweek  # Monday=0, Sunday=6
    df['day'] = df['DATETIME_UTC'].dt.day
    df['month'] = df['DATETIME_UTC'].dt.month
    df['year'] = df['DATETIME_UTC'].dt.year
    df['is_weekend'] = df['day_of_week'] >= 5

    # Define season mapping
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'

    df['season'] = df['month'].apply(get_season)

    print("Temporal features added:")
    display(df[['hour', 'day_of_week', 'day', 'month', 'year', 'is_weekend', 'season']].head())

Temporal features added:


,hour,day_of_week,day,month,year,is_weekend,season
0,5,0,30,6,2014,False,Summer
1,1,0,17,3,2014,False,Spring
2,2,1,26,11,2013,False,Fall
3,13,5,26,10,2013,True,Fall
4,23,2,26,6,2013,False,Summer


### 5. Verify Temporal Feature Data Types

Check that the new temporal features have correct data types.

In [6]:
if 'df' in locals():
    print(df[['hour', 'day_of_week', 'day', 'month', 'year', 'is_weekend', 'season']].dtypes)
    print("\nAny null values in temporal features:")
    print(df[['hour', 'day_of_week', 'month', 'season']].isnull().sum())

hour            int32
day_of_week     int32
day             int32
month           int32
year            int32
is_weekend       bool
season         object
dtype: object

Any null values in temporal features:
hour           0
day_of_week    0
month          0
season         0
dtype: int64


**Next Steps (Part 2):** Temporal feature engineering (hour, day of week, month, season, etc.) in this notebook.

### 6. Missing Value Handling

Analyze and impute missing values for categorical features such as `WEATHER`, `LIGHT`, `MAIN_CAUSE`, `COLLISION_TYPE`, and `REPORTING_AGENCY`. Fill missing entries with 'Unknown' and document the strategy.

In [8]:
# List of categorical columns to check and impute
cat_cols = ['WEATHER', 'LIGHT', 'MAIN_CAUSE', 'COLLISION_TYPE', 'REPORTING_AGENCY']

# Display missing value counts before imputation
print("Missing values before imputation:")
print(df[cat_cols].isnull().sum())

# Impute missing categorical values with 'Unknown'
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# Verify missing values after imputation
print("\nMissing values after imputation:")
print(df[cat_cols].isnull().sum())

Missing values before imputation:
WEATHER             21768
LIGHT               21768
MAIN_CAUSE          21768
COLLISION_TYPE          0
REPORTING_AGENCY        0
dtype: int64

Missing values after imputation:
WEATHER             0
LIGHT               0
MAIN_CAUSE          0
COLLISION_TYPE      0
REPORTING_AGENCY    0
dtype: int64


**Strategy:** Imputed missing categorical entries with 'Unknown' to preserve all records and explicitly flag missing data. This approach avoids bias from mode imputation and maintains consistency.

### 7. Next Steps (Part 4)

Encode categorical features (`ROAD`, `MAIN_CAUSE`, `COLLISION_TYPE`, `WEATHER`, `LIGHT`) using One-Hot Encoding or other suitable techniques. Ensure documentation of choices and rationale.

### 8. Categorical Feature Encoding

Encode categorical features (`ROAD`, `MAIN_CAUSE`, `COLLISION_TYPE`, `WEATHER`, `LIGHT`, `REPORTING_AGENCY`) using One-Hot Encoding to convert them into numeric indicators.

In [9]:
# Perform One-Hot Encoding on selected categorical columns
cat_encode_cols = ['ROAD', 'MAIN_CAUSE', 'COLLISION_TYPE', 'WEATHER', 'LIGHT', 'REPORTING_AGENCY']
encode_list = [col for col in cat_encode_cols if col in df.columns]
df = pd.get_dummies(df, columns=encode_list, prefix=encode_list, dummy_na=False)
print(f"After encoding categorical features, dataframe has {df.shape[1]} columns")

After encoding categorical features, dataframe has 41 columns


### 9. Verify Encoding Results

Inspect a sample of the newly created dummy columns to ensure correct encoding.

In [10]:
# Display a sample of the encoded dummy columns
encoded_cols = [col for col in df.columns if any(col.startswith(pref + '_') for pref in encode_list)]
display(df[encoded_cols].head())
print(f"Total encoded features: {len(encoded_cols)}")

,ROAD_EDSA,MAIN_CAUSE_Human error,MAIN_CAUSE_Other (see description),MAIN_CAUSE_Road defect,MAIN_CAUSE_Unknown,MAIN_CAUSE_Vehicle defect,COLLISION_TYPE_Angle Impact,COLLISION_TYPE_Head-On,COLLISION_TYPE_Hit Object,COLLISION_TYPE_Multiple,...,WEATHER_partly-cloudy-day,WEATHER_partly-cloudy-night,WEATHER_rain,LIGHT_Unknown,LIGHT_day,LIGHT_dusk,LIGHT_night,REPORTING_AGENCY_MMDA Metrobase,REPORTING_AGENCY_MMDA Road Safety Unit,REPORTING_AGENCY_Other
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False


Total encoded features: 29


### 10. Next Steps (Part 5)

Proceed to scale numerical features (`Y`, `X`), parse `DESC` for derived features, drop or transform as needed, and save the fully preprocessed DataFrame for downstream modeling.

### 11. Numerical Feature Planning & `DESC` Parsing

Plan for numerical scaling in the modeling pipeline (e.g., fit scaler on `X_train`).
Here we derive basic features from the `DESC` text column: word count and presence of the word "collision".

In [11]:
# Parse basic features from DESC
if 'df' in locals() and 'DESC' in df.columns:
    df['desc_word_count'] = df['DESC'].astype(str).str.split().apply(len)
    df['desc_contains_collision'] = df['DESC'].str.contains('collision', case=False, na=False).astype(int)
    # Drop original DESC column
    df.drop(columns=['DESC'], inplace=True)
    print("Derived DESC features and dropped original DESC column.")
    display(df[['desc_word_count', 'desc_contains_collision']].head())

Derived DESC features and dropped original DESC column.


,desc_word_count,desc_contains_collision
0,30,1
1,38,1
2,30,1
3,31,1
4,32,1


### 12. Save Fully Preprocessed DataFrame

Save the cleaned and feature-engineered DataFrame for use in downstream modeling notebooks.

In [12]:
# Ensure output directory exists and save
import os
os.makedirs('../data/processed', exist_ok=True)
df.to_csv('../data/processed/preprocessed_data.csv', index=False)
print("Preprocessed data saved to '../data/processed/preprocessed_data.csv'")

Preprocessed data saved to '../data/processed/preprocessed_data.csv'
